In [ ]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, ranksums

def get_intersection_item(sets):
    ref = set(sets[0])
    for item in sets[1:]:
        ref = ref.intersection(item)
    return sorted(ref)

def get_union_item(sets):
    ref = set(sets[0])
    for item in sets[1:]:
        ref = ref.union(item)
    return sorted(ref)

def get_degree_feature(adj, nodes):
    ex_adj = pd.DataFrame(np.zeros((len(nodes), len(nodes))), index=nodes, columns=nodes)
    index = adj.index.intersection(nodes)
    ex_adj.loc[index, index] = adj.loc[index, index]
    degree = ex_adj.sum(axis=0)
    return degree

def get_similary(data, method='spearman'):
    if method=='spearman':
        func = spearmanr
    elif method=="ranksum":
        func = ranksums
    columns = data.columns
    values = pd.DataFrame(np.zeros((len(columns), len(columns))), index=columns, columns=columns)
    p_values = pd.DataFrame(np.zeros((len(columns), len(columns))), index=columns, columns=columns)
    for row in columns:
        for col in columns:
            value, pvalue = func(data[row].values, data[col].values)
            values.loc[row, col] = value
            p_values.loc[row, col] = pvalue
    return values, p_values

def summary_graph(graph_dir):
    ans = []
    for cell_type in os.listdir(graph_dir):
        node_file = os.path.join(graph_dir, cell_type, "origin_graph", 'adj.txt')
        x_node_file = os.path.join(graph_dir, cell_type, "x_graph.txt")
        tmp = {"cell_type": cell_type}
        if os.path.exists(node_file):
            adj = pd.read_csv(node_file, index_col=0, sep='\t')
            node_num = adj.shape[0]
            edge_num = adj.values.sum()
            tmp['origin_node'] = node_num
            tmp['origin_edge'] = edge_num
            tmp['origin_density'] = edge_num/(node_num**2)
        if os.path.exists(x_node_file):
            adj = pd.read_csv(x_node_file, index_col=0, sep='\t')
            node_num = adj.shape[0]
            edge_num = adj.values.sum()
            tmp['share_node'] = node_num
            tmp['share_edge'] = edge_num
            tmp['share_density'] = edge_num/(node_num**2)
        ans.append(tmp)
    return pd.DataFrame(ans)

def summary_data(data_dir):
    ans = []
    for cell_type in os.listdir(data_dir):
        atac_dir = os.path.join(data_dir, cell_type, 'atac')
        scrna_dir = os.path.join(data_dir, cell_type, 'scrna')
        if os.path.exists(atac_dir):
            barcodes = pd.read_csv(os.path.join(atac_dir, "barcodes.tsv"), header=None)
            peaks = pd.read_csv(os.path.join(atac_dir, "peaks.tsv"), header=None)
            tmp = {"cell_type": cell_type,
                  "data_type": 'ATAC',
                  "cell_num": len(barcodes),
                  "feature_num": len(peaks)}
            ans.append(tmp)
        if os.path.exists(scrna_dir):
            barcodes = pd.read_csv(os.path.join(scrna_dir, "barcodes.tsv"), header=None)
            genes = pd.read_csv(os.path.join(scrna_dir, "genes.tsv"), header=None)
            tmp = {"cell_type": cell_type,
                  "data_type": 'scRNA',
                  "cell_num": len(barcodes),
                  "feature_num": len(genes)}
            ans.append(tmp)
    return pd.DataFrame(ans)


def get_degree(graph_dir, graph_type='origin_graph'):
    graphs = {}
    for cell_type in os.listdir(graph_dir):
        if graph_type=="origin_graph":
            node_file = os.path.join(graph_dir, cell_type, graph_type, 'adj.txt')
        else:
            node_file = os.path.join(graph_dir, cell_type, "x_graph.txt")
        if os.path.exists(node_file):
            adj = pd.read_csv(node_file, index_col=0, sep='\t')
            graphs[cell_type.split("_")[0]] = adj
    union_genes = get_union_item([value.index for value in graphs.values()])
    union_degrees = pd.DataFrame({key:get_degree_feature(g, union_genes) for key,g in graphs.items()})
    return union_degrees

def plot_degree(degrees, bins=30):
    max_degree = degrees.values.max()
    bins = np.arange(0, max_degree, max_degree//bins)
    f, axs = plt.subplots(len(degree.columns), sharex=True, figsize=(20,20))
    for i, col in enumerate(degree.columns):
        ax = sns.histplot(degree[col].values, ax=axs[i], bins=bins)
        ax.set_title(col)
#         plt.xlabel("degree")
#         plt.ylabel("# of TFs")
#         axs[i].title.set_text(col)

def summary_degree_shared(graph_dir, graph_type='origin_graph', method='spearman'):
    graphs = {}
    for cell_type in os.listdir(graph_dir):
        if graph_type=="origin_graph":
            node_file = os.path.join(graph_dir, cell_type, graph_type, 'adj.txt')
        else:
            node_file = os.path.join(graph_dir, cell_type, "x_graph.txt")
        if os.path.exists(node_file):
            adj = pd.read_csv(node_file, index_col=0, sep='\t')
            graphs[cell_type] = adj
    columns = list(graphs.keys())
    ans = pd.DataFrame(np.zeros((len(graphs), len(graphs))), index=columns, columns=columns)
    for row in columns:
        for col in columns:
            ans.loc[row, col] = len(graphs[row].index.intersection(graphs[col].index))
    return ans


def summary_graph_similary(graph_dir, graph_type='origin_graph', method='spearman'):
    graphs = {}
    for cell_type in os.listdir(graph_dir):
        if graph_type=="origin_graph":
            node_file = os.path.join(graph_dir, cell_type, graph_type, 'adj.txt')
        else:
            node_file = os.path.join(graph_dir, cell_type, "x_graph.txt")
        if os.path.exists(node_file):
            adj = pd.read_csv(node_file, index_col=0, sep='\t')
            graphs[cell_type] = adj
    
    union_genes = get_union_item([value.index for value in graphs.values()])
    unique_genes = get_intersection_item([value.index for value in graphs.values()])
    print(f"union_genes:{len(union_genes)}")
    print(f"unique_gene:{len(unique_genes)}")
    union_degrees = pd.DataFrame({key:get_degree_feature(g, union_genes) for key,g in graphs.items()})
    unique_degrees = pd.DataFrame({key:get_degree_feature(g, unique_genes) for key,g in graphs.items()})
    
    return {"union": get_similary(union_degrees, method=method),
           "unique": get_similary(unique_degrees, method=method)}



# 参考
* https://stuartlab.org/signac/articles/pbmc_vignette.html
* https://satijalab.org/seurat/articles/atacseq_integration_vignette
* https://satijalab.org/seurat/articles/pbmc3k_tutorial
* https://satijalab.github.io/azimuth/articles/run_azimuth_tutorial.html

## hg19.fa
https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz

## hg38.fa
https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

## HOCOMOCO10
https://hocomoco10.autosome.org/final_bundle/HUMAN/mono/HOCOMOCOv10_HUMAN_mono_meme_format.meme

# AD_GSE174367

## code
* GSE174367_scRNA_Process.ipynb
* GSE174367_ATAC_Process.ipynb

## downloaded data file

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE174367

* GSE174367_snATAC-seq_cell_meta.csv
* GSE174367_snATAC-seq_filtered_peak_bc_matrix.h5
* GSE174367_snRNA-seq_cell_meta.csv
* GSE174367_snRNA-seq_filtered_feature_bc_matrix.h5

#### 统计ATAC和scRNA各细胞类型

In [ ]:
df = summary_data("./AD_GSE174367/ad_sep_data")
list(df.groupby('data_type'))

#### 统计构建的GRN网络信息
origin: 使用ATAC数据构建

share: 在origin的基础上使用scRNA高方差的gene构建

In [ ]:
summary_graph("./AD_GSE174367/graph/hg38_HOCOMOCOv11")

#### 不同细胞间共享的节点数

In [ ]:
summary_degree_shared("./AD_GSE174367/graph/hg38_HOCOMOCOv11")

#### 不同细胞间的度相似性
union: 不同细胞的GRN节点取并集，缺失的节点度为0

unique: 不同细胞的GRN节点取交集

In [ ]:
df = summary_graph_similary("./AD_GSE174367/graph/hg38_HOCOMOCOv11")['union'][0]
plt.figure(figsize=(10,10))
sns.heatmap(df)
df

In [ ]:
df = summary_graph_similary("./AD_GSE174367/graph/hg38_HOCOMOCOv11")['unique'][0]
plt.figure(figsize=(10,10))
sns.heatmap(df)
df

In [ ]:
degree = get_degree("./AD_GSE174367/graph/hg38_HOCOMOCOv11")
plot_degree(degree, bins=30)
degree

# Lung_GSM4508936

## code
* Lung_preprocess.ipynb

## downloaded data file

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM4508936

* GSM4508936_lung_filtered.seurat.RDS

#### 统计ATAC和scRNA各细胞类型

In [ ]:
df = summary_data("./Lung_GSM4508936/lung_sep_data")
list(df.groupby('data_type'))

#### 统计构建的GRN网络信息
origin: 使用ATAC数据构建

share: 在origin的基础上使用scRNA高方差的gene构建

In [ ]:
summary_graph("./Lung_GSM4508936/graph/hg19_HOCOMOCOv11")

#### 不同细胞间共享的节点数

In [ ]:
summary_degree_shared("./Lung_GSM4508936/graph/hg19_HOCOMOCOv11")

#### 不同细胞间的度相似性
union: 不同细胞的GRN节点取并集，缺失的节点度为0

unique: 不同细胞的GRN节点取交集

In [ ]:
df = summary_graph_similary("./Lung_GSM4508936/graph/hg19_HOCOMOCOv11")['union'][0]
plt.figure(figsize=(10,10))
sns.heatmap(df)
df

In [ ]:
df = summary_graph_similary("./Lung_GSM4508936/graph/hg19_HOCOMOCOv11")['unique'][0]
plt.figure(figsize=(10,10))
sns.heatmap(df)
df

In [ ]:
degree = get_degree("./Lung_GSM4508936/graph/hg19_HOCOMOCOv11")
plot_degree(degree, bins=40)
degree

# PBMC

## code
* pbmc_script.R
* RunAzimuthLocal.R
* PBMC10k_ATAC_Process.ipynb
* PBMC8k_scRNA_Process.ipynb

## downloaded data file

### ATAC
https://support.10xgenomics.com/single-cell-atac/datasets/1.2.0/atac_v1_pbmc_10k
* atac_v1_pbmc_10k_fragments.tsv.gz
* atac_v1_pbmc_10k_fragments.tsv.gz.tbi
* atac_v1_pbmc_10k_singlecell.csv
* atac_v1_pbmc_10k_filtered_peak_bc_matrix.tar.gz

### scRNA
https://support.10xgenomics.com/single-cell-gene-expression/datasets/2.1.0/pbmc8k

* pbmc8k_filtered_gene_bc_matrices.tar.gz

### Azimuth

#### scRNA reference file
https://zenodo.org/record/4546839 
* idx.annoy
* ref.Rds

#### ATAC reference file
https://zenodo.org/record/7770389#.ZFhDx-zMLHo 
* ext.Rds

#### homologs.rds
https://seurat.nygenome.org/azimuth/references/homologs.rds 

#### 统计ATAC和scRNA各细胞类型

In [ ]:
df = summary_data("./PBMC/pbmc_sep_data")
list(df.groupby('data_type'))

#### 统计构建的GRN网络信息
origin: 使用ATAC数据构建

share: 在origin的基础上使用scRNA高方差的gene构建

In [ ]:
summary_graph("./PBMC/graph/hg19_HOCOMOCOv11")

#### 不同细胞间共享的节点数

In [ ]:
summary_degree_shared("./PBMC/graph/hg19_HOCOMOCOv11")

#### 不同细胞间的度相似性
union: 不同细胞的GRN节点取并集，缺失的节点度为0

unique: 不同细胞的GRN节点取交集

In [ ]:
df = summary_graph_similary("./PBMC/graph/hg19_HOCOMOCOv11")['union'][0]
plt.figure(figsize=(10,10))
sns.heatmap(df)
df

In [ ]:
df = summary_graph_similary("./PBMC/graph/hg19_HOCOMOCOv11")['unique'][0]
plt.figure(figsize=(10,10))
sns.heatmap(df)
df

In [ ]:
degree = get_degree("./PBMC/graph/hg19_HOCOMOCOv11")
plot_degree(degree, bins=40)
degree

# PBMC Signac
## Code
* pbmc_vignette.Rmd 
    * https://stuartlab.org/signac/articles/pbmc_vignette.html
* PBMC_signac_ATAC_process.ipynb
* PBMC_signac_scRNA_process.ipynb

## downloaded data file
### ATAC
https://www.10xgenomics.com/resources/datasets/10-k-peripheral-blood-mononuclear-cells-pbm-cs-from-a-healthy-donor-1-standard-1-0-1
* atac_v1_pbmc_10k_fragments.tsv.gz
* atac_v1_pbmc_10k_fragments.tsv.gz.tbi
* atac_v1_pbmc_10k_singlecell.csv
* atac_v1_pbmc_10k_filtered_peak_bc_matrix.tar.gz

### scRNA
https://support.10xgenomics.com/single-cell-gene-expression/datasets/3.0.0/pbmc_10k_v3

* https://signac-objects.s3.amazonaws.com/pbmc_10k_v3.rds

#### 统计ATAC和scRNA各细胞类型

In [ ]:
df = summary_data("./PBMC_signac/pbmc_signac_sep_data")
list(df.groupby('data_type'))

#### 统计构建的GRN网络信息
origin: 使用ATAC数据构建

share: 在origin的基础上使用scRNA高方差的gene构建

In [ ]:
summary_graph("./PBMC_signac/graph/hg19_HOCOMOCOv11")

#### 不同细胞间共享的节点数

In [ ]:
summary_degree_shared("./PBMC_signac/graph/hg19_HOCOMOCOv11")

#### 不同细胞间的度相似性
union: 不同细胞的GRN节点取并集，缺失的节点度为0

unique: 不同细胞的GRN节点取交集

In [ ]:
df = summary_graph_similary("./PBMC_signac/graph/hg19_HOCOMOCOv11")['union'][0]
plt.figure(figsize=(10,10))
sns.heatmap(df)
df

In [ ]:
df = summary_graph_similary("./PBMC_signac/graph/hg19_HOCOMOCOv11")['unique'][0]
plt.figure(figsize=(10,10))
sns.heatmap(df)
df

In [ ]:
degree = get_degree("./PBMC_signac/graph/hg19_HOCOMOCOv11")
plot_degree(degree, bins=30)
degree

# AD Epigenome
https://compbio.mit.edu/ad_epigenome/
## Code

* preprocess_scrna_01.ipynb
* earlyAD_scRNA_Process.ipynb
* lateAD_scRNA_Process.ipynb
* nonAD_scRNA_Process.ipynb


* preprocess_atac_01.ipynb
* earlyAD_ATAC_Process.ipynb
* lateAD_ATAC_Process.ipynb
* nonAD_ATAC_Process.ipynb

## downloaded data file
### ATAC
https://personal.broadinstitute.org/bjames/AD_snATAC/TSS6_highQC/PeakMatrix.TSS6.cleaned.rds


### scRNA
https://personal.broadinstitute.org/bjames/AD_snATAC/RNA/RNA.h5ad

#### 统计ATAC和scRNA各细胞类型

In [ ]:
df = summary_data("./AD_epigenome/earlyAD/sep_data")
list(df.groupby('data_type'))

In [ ]:
df = summary_data("./AD_epigenome/lateAD/sep_data")
list(df.groupby('data_type'))

In [ ]:
df = summary_data("./AD_epigenome/nonAD/sep_data")
list(df.groupby('data_type'))

#### 统计构建的GRN网络信息
origin: 使用ATAC数据构建

share: 在origin的基础上使用scRNA高方差的gene构建

In [ ]:
summary_graph("./AD_epigenome/nonAD/graph/hg38_HOCOMOCOv11")

In [ ]:
summary_graph("./AD_epigenome/lateAD/graph/hg38_HOCOMOCOv11")

In [ ]:
summary_graph("./AD_epigenome/earlyAD/graph/hg38_HOCOMOCOv11")

In [ ]:
summary_graph("./AD_epigenome/earlyAD/sep_data2/graph_90/hg38_HOCOMOCOv11")

In [ ]:
summary_graph("./AD_epigenome/earlyAD/sep_data2/graph/hg38_HOCOMOCOv11")

#### 不同细胞间共享的节点数

In [ ]:
summary_degree_shared("./AD_epigenome/earlyAD/graph/hg38_HOCOMOCOv11")

#### 不同细胞间的度相似性
union: 不同细胞的GRN节点取并集，缺失的节点度为0

unique: 不同细胞的GRN节点取交集

In [ ]:
df = summary_graph_similary("./AD_epigenome/earlyAD/graph/hg38_HOCOMOCOv11")['union'][0]
plt.figure(figsize=(10,10))
sns.heatmap(df)
df

In [ ]:
df = summary_graph_similary("./AD_epigenome/earlyAD/graph/hg38_HOCOMOCOv11")['unique'][0]
plt.figure(figsize=(10,10))
sns.heatmap(df)
df

In [ ]:
degree = get_degree("./AD_epigenome/earlyAD/graph/hg38_HOCOMOCOv11")
plot_degree(degree, bins=20)
degree

# SEA-AD
https://registry.opendata.aws/allen-sea-ad-atlas/

## Code

* preprocess_scrna_01.ipynb
* earlyAD_scRNA_Process.ipynb
* lateAD_scRNA_Process.ipynb
* nonAD_scRNA_Process.ipynb


* preprocess_atac_01.ipynb
* earlyAD_ATAC_Process.ipynb
* lateAD_ATAC_Process.ipynb
* nonAD_ATAC_Process.ipynb

## downloaded data file
### ATAC
https://sea-ad-single-cell-profiling.s3.amazonaws.com/MTG/ATACseq/SEAAD_MTG_ATACseq_final-nuclei.2023-05-08.h5ad

### scRNA
https://sea-ad-single-cell-profiling.s3.amazonaws.com/MTG/RNAseq/SEAAD_MTG_RNAseq_final-nuclei.2023-05-05.h5ad